In [1]:
# Can be autimated across quarter if needed

import pandas as pd
import numpy as np
import datetime
import os
import glob


def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [2]:
# Fix fisrt week
# last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2)
last_saturday=datetime.date(2018,12,15)

First_week_ending_Q4_2018=datetime.date(2018,11,10)
Q4_2018_weeks=[First_week_ending_Q4_2018+datetime.timedelta(days=x*7) for x in range(13)]


Q4_2018_weeks=[x for x in Q4_2018_weeks if x <=last_saturday]

Q4_2017_weeks=[x-datetime.timedelta(days=52*7) for x in Q4_2018_weeks]
len_Q4_weeks=len(Q4_2018_weeks)
Cum_nth_week=int((last_saturday-last_saturday).days/7)+1

write_folder="/home/jian/celery/Quadrants_Dashboard/output/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)
    
Cum_nth_week=int((last_saturday-First_week_ending_Q4_2018).days/7)+1

previouse_folder="/home/jian/celery/Quadrants_Dashboard/output/output_"+str(last_saturday-datetime.timedelta(days=7))+"/"

In [3]:
# Use the weekly all wide/long output as the starting trigger
# Than tap fater existing

In [4]:

sales_all=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_"+str(last_saturday)+".csv",dtype=str,usecols=['location_id','sales','transactions','week_end_date'])
sales_all['sales']=sales_all['sales'].astype(float)
sales_all['transactions']=sales_all['transactions'].astype(float).astype(int)
sales_all=sales_all[sales_all['sales']>0]
sales_all_Q4_2017=sales_all[sales_all['week_end_date'].isin([str(x) for x in Q4_2017_weeks])]
sales_all_Q4_2018=sales_all[sales_all['week_end_date'].isin([str(x) for x in Q4_2018_weeks])]

sales_all_Q4_both_year=sales_all_Q4_2017.append(sales_all_Q4_2018)

inclusion_stores=sales_all_Q4_both_year.groupby(['location_id'])['sales'].count().to_frame().reset_index().rename(columns={"sales":"week_count"})
inclusion_stores=inclusion_stores[inclusion_stores['week_count']==2*len_Q4_weeks]
set_inclusion_stores=set(inclusion_stores['location_id'].unique().tolist())

sales_all_2017Q4_inclusions=sales_all_Q4_2017[sales_all_Q4_2017['location_id'].isin(set_inclusion_stores)]
sales_all_2017Q4_inclusions=sales_all_2017Q4_inclusions.rename(columns={"sales":"sales_2017","transactions":"trans_2017"})
sales_all_2018Q4_inclusions=sales_all_Q4_2018[sales_all_Q4_2018['location_id'].isin(set_inclusion_stores)]
sales_all_2018Q4_inclusions=sales_all_2018Q4_inclusions.rename(columns={"sales":"sales_2018","transactions":"trans_2018"})
sales_all_2018Q4_inclusions['week_end_date']=sales_all_2018Q4_inclusions['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
sales_all_2017Q4_inclusions['week_end_date']=sales_all_2017Q4_inclusions['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()+datetime.timedelta(days=52*7))

sales_all_Q4_inclusions=pd.merge(sales_all_2017Q4_inclusions,sales_all_2018Q4_inclusions,on=["location_id","week_end_date"],how="outer")
sales_all_Q4_inclusions['Quadrant']=np.nan
sales_all_Q4_inclusions=sales_all_Q4_inclusions[['location_id','week_end_date','sales_2017','sales_2018','trans_2017','trans_2018','Quadrant']]


exclusion_stores_2017Q4=sales_all_Q4_2017[~sales_all_Q4_2017['location_id'].isin(set_inclusion_stores)][['location_id']].drop_duplicates()
exclusion_stores_2018Q4=sales_all_Q4_2018[~sales_all_Q4_2018['location_id'].isin(set_inclusion_stores)][['location_id']].drop_duplicates()



sales_all_Q4_inclusions_Cum=sales_all_Q4_inclusions.groupby(['location_id'])['sales_2017','sales_2018','trans_2017','trans_2018'].sum().reset_index()
sales_all_Q4_inclusions_Cum['Total_Sales_YoY']=sales_all_Q4_inclusions_Cum['sales_2018']/sales_all_Q4_inclusions_Cum['sales_2017']-1
sales_all_Q4_inclusions_Cum['Total_Sales_YoY']=sales_all_Q4_inclusions_Cum['Total_Sales_YoY'].apply(lambda x: np.round(x,4))


exclusion_stores_2018Q4.to_csv(write_folder+"Exclusion_of_2018_Q4.csv",index=False)
exclusion_stores_2017Q4.to_csv(write_folder+"Exclusion_of_2017_Q4.csv",index=False)

In [5]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in Q4_2018_weeks]
recent_daily_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("aily" in x) & (".txt" in x) ]
    recent_daily_list=recent_daily_list+list_recent
recent_daily_df=pd.DataFrame({"path":recent_daily_list,"date":[datetime.datetime.strptime(x.split("MediaStormDailySales")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_daily_list]},index=[x for x in range(len(recent_daily_list))])


In [6]:
list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/")) if ("aily" in x) & (".txt" in x) ]
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
del df_1_after_201806['index']

list_2_before_201806=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data/"))
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-1].split("_")[3].split(".")[0],"%Y-%m-%d").date() for x in list_2_before_201806]
df_2_before_201806=pd.DataFrame({"date":folder_date,"path":list_2_before_201806},index=[x for x in range(len(list_2_before_201806))])
df_2_before_201806['date'].apply(lambda x: x.weekday()).unique()
df_2_before_201806=df_2_before_201806.sort_values('date').reset_index()
del df_2_before_201806['index']
df_2_before_201806['date'].apply(lambda x: x.weekday()).unique()

df_Q4_2017=df_2_before_201806[df_2_before_201806['date'].isin(Q4_2017_weeks)]
df_Q4_2018=df_1_after_201806[df_1_after_201806['date'].isin(Q4_2018_weeks)]
df_Q4_2018=df_Q4_2018.append(recent_daily_df)

In [7]:
def add_week_end_dt(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y

def count_unique(x):
    y=len(set(x))
    return y

In [8]:
agg_2017_ids_by_week=pd.read_csv(previouse_folder+"cum_shoppers_id_by_store_2017Q4.csv",dtype=str)
try:
    file_path=df_Q4_2017.loc[df_Q4_2017['date']==last_saturday-datetime.timedelta(days=52*7),'path'].get_values()[0]
# Will occur error if not the day
except:
    print("Error 2017 last week path")
df=pd.read_table(file_path,dtype=str,sep="|",usecols=['location_id','transaction_dt','customer_id_hashed'])
df=df[df['location_id'].isin(set_inclusion_stores)]
df=df[~pd.isnull(df['customer_id_hashed'])].drop_duplicates()
agg_2017_ids_by_week=agg_2017_ids_by_week.append(df)
print(datetime.datetime.now())
print(agg_2017_ids_by_week.shape)
# To add: last week cum
agg_2017_ids_by_week=agg_2017_ids_by_week[['location_id','customer_id_hashed']].drop_duplicates()
print(agg_2017_ids_by_week.shape)
agg_2017_ids_by_week.to_csv(write_folder+"cum_shoppers_id_by_store_2017Q4.csv",index=False)

cum_shopper_2017_count=agg_2017_ids_by_week.groupby(['location_id'])['customer_id_hashed'].count().to_frame().reset_index()
cum_shopper_2017_count=cum_shopper_2017_count.rename(columns={"customer_id_hashed":"shoppers_cum_2017Q4"})

2019-01-03 12:31:52.715840
(6798675, 3)
(6014302, 2)


In [9]:
cum_daily_summary_sales_trans=pd.read_csv(previouse_folder+"cum_daily_sales.csv",dtype=str)
cum_daily_summary_sales_trans=cum_daily_summary_sales_trans[cum_daily_summary_sales_trans['location_id'].isin(set_inclusion_stores)]
del cum_daily_summary_sales_trans['shoppers_cum_2018Q4']
del cum_daily_summary_sales_trans['Quadrant']
del cum_daily_summary_sales_trans['cum_weeks']
cum_daily_summary_sales_trans['Rewards_sales']=cum_daily_summary_sales_trans['Rewards_sales'].astype(float)
cum_daily_summary_sales_trans['Rewards_trans']=cum_daily_summary_sales_trans['Rewards_trans'].astype(int)
cum_daily_summary_sales_trans['Non_rewards_sales']=cum_daily_summary_sales_trans['Non_rewards_sales'].astype(float)
cum_daily_summary_sales_trans['Non_rewards_trans']=cum_daily_summary_sales_trans['Non_rewards_trans'].astype(float)


agg_2018_ids_by_week=pd.read_csv(previouse_folder+"cum_shoppers_id_by_store_2018Q4.csv",dtype=str)

try:
    file_path=df_Q4_2018.loc[df_Q4_2018['date']==last_saturday,'path'].get_values()[0]
# Will occur error if not the day
except:
    print("Error 2018 last week path")

df=pd.read_table(file_path,dtype=str,sep="|",usecols=['location_id', 'transaction_dt', 'transaction_id','customer_id_hashed', 'subclass_transaction_amt'])
df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
df['transaction_dt']=df['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
df['week_end_dt']=df['transaction_dt'].apply(lambda x: add_week_end_dt(x))

df_checking_sales=df.copy().rename(columns={"week_end_dt":"week_end_date"})
df_checking_sales['week_end_date']=df_checking_sales['week_end_date'].astype(str)
df_checking_sales=df_checking_sales.groupby(['week_end_date','location_id'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"sales_daily_data"})
df_checking_sales=pd.merge(df_checking_sales,sales_all,on=['week_end_date','location_id'],how="left")
df_checking_sales['variance_sales']=(df_checking_sales['sales_daily_data']-df_checking_sales['sales'])/df_checking_sales['sales']
df_checking_sales['variance_sales']=df_checking_sales['variance_sales'].apply(lambda x: np.round(x,4))

df=df[df['location_id'].isin(set_inclusion_stores)]

df_dedup=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','week_end_dt']].drop_duplicates()

df_all_sales=df.groupby(['location_id','week_end_dt'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"sales"})
df_all_trans=df_dedup.groupby(['location_id','week_end_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
df_all=pd.merge(df_all_sales,df_all_trans,on=['location_id','week_end_dt'],how="outer")


df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
df_dedup_non_rewards=df_dedup[pd.isnull(df_dedup['customer_id_hashed'])]
df_non_rewards_sales=df_non_rewards.groupby(['location_id','week_end_dt'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"Non_rewards_sales"})
df_non_rewards_trans=df_dedup_non_rewards.groupby(['location_id','week_end_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"Non_rewards_trans"})
df_non_rewards=pd.merge(df_non_rewards_sales,df_non_rewards_trans,on=['location_id','week_end_dt'],how="outer")
del df_dedup_non_rewards

df_rewards=df[~pd.isnull(df['customer_id_hashed'])]
# df_dedup_rewards=df_rewards[['location_id','transaction_dt','transaction_id','week_end_dt']].drop_duplicates()
df_dedup_rewards=df_dedup[~pd.isnull(df_dedup['customer_id_hashed'])]
df_rewards_sales=df_rewards.groupby(['location_id','week_end_dt'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"Rewards_sales"})
df_rewards_trans=df_dedup_rewards.groupby(['location_id','week_end_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"Rewards_trans"})
df_rewards=pd.merge(df_rewards_sales,df_rewards_trans,on=['location_id','week_end_dt'],how="outer")
del df_dedup_rewards

df_daily_summary_sales_trans=pd.merge(df_rewards,df_non_rewards,on=['location_id','week_end_dt'],how="outer")
df_daily_summary_sales_trans=df_daily_summary_sales_trans.fillna(0)
#1
cum_daily_summary_sales_trans=cum_daily_summary_sales_trans.append(df_daily_summary_sales_trans)
cum_daily_summary_sales_trans.to_csv(write_folder+"df_2018_Q4_all_weeks_by_store_week.csv",index=False)

df_rewards_ids=df[['location_id','transaction_dt','customer_id_hashed']].drop_duplicates()
df_rewards_ids=df_rewards_ids[~pd.isnull(df_rewards_ids['customer_id_hashed'])].reset_index()
del df_rewards_ids['index']
#2
agg_2018_ids_by_week=agg_2018_ids_by_week.append(df_rewards_ids)
agg_2018_ids_by_week=agg_2018_ids_by_week[['location_id','customer_id_hashed']].drop_duplicates()
agg_2018_ids_by_week.to_csv(write_folder+"cum_shoppers_id_by_store_2018Q4.csv",index=False)

cum_shopper_2018_count=agg_2018_ids_by_week.groupby(['location_id'])['customer_id_hashed'].count().to_frame().reset_index()
cum_shopper_2018_count=cum_shopper_2018_count.rename(columns={"customer_id_hashed":"shoppers_cum_2018Q4"})

In [10]:
sales_all_Q4_inclusions_Cum=pd.merge(sales_all_Q4_inclusions_Cum,cum_shopper_2018_count,on="location_id",how="inner")
sales_all_Q4_inclusions_Cum=pd.merge(sales_all_Q4_inclusions_Cum,cum_shopper_2017_count,on="location_id",how="inner")
sales_all_Q4_inclusions_Cum['shopper_growth']=sales_all_Q4_inclusions_Cum['shoppers_cum_2018Q4']-sales_all_Q4_inclusions_Cum['shoppers_cum_2017Q4']

In [11]:
#2018 Q2 Quadrant shopper growth threshold >=889
sales_all_Q4_inclusions_Cum['Quadrant']=np.where((sales_all_Q4_inclusions_Cum['Total_Sales_YoY']>=0) & (sales_all_Q4_inclusions_Cum['shopper_growth']>=889),"Quadrant I",
                                                 np.where((sales_all_Q4_inclusions_Cum['Total_Sales_YoY']<0) & (sales_all_Q4_inclusions_Cum['shopper_growth']>=889),"Quadrant II",
                                                          np.where((sales_all_Q4_inclusions_Cum['Total_Sales_YoY']<0) & (sales_all_Q4_inclusions_Cum['shopper_growth']<889),"Quadrant III",
                                                                  np.where((sales_all_Q4_inclusions_Cum['Total_Sales_YoY']>=0) & (sales_all_Q4_inclusions_Cum['shopper_growth']<889),"Quadrant IV","Error")
                                                                  )
                                                         )
                                                 )

df_store_quadrant=sales_all_Q4_inclusions_Cum[['location_id','Quadrant']]

In [12]:
sales_all_Q4_inclusions_Cum_by_Quadrant=sales_all_Q4_inclusions_Cum.groupby(['Quadrant'])['sales_2017','sales_2018','trans_2017','trans_2018','shoppers_cum_2018Q4','shoppers_cum_2017Q4'].sum().reset_index()
df_count_store=sales_all_Q4_inclusions_Cum.groupby(['Quadrant'])['location_id'].count().to_frame().reset_index().rename(columns={"location_id":"store_count"})
sales_all_Q4_inclusions_Cum_by_Quadrant=pd.merge(sales_all_Q4_inclusions_Cum_by_Quadrant,df_count_store,on=['Quadrant'],how="inner")
sales_all_Q4_inclusions_Cum_by_Quadrant['cum_weeks']=Cum_nth_week


In [13]:
cum_daily_summary_sales_trans=cum_daily_summary_sales_trans.groupby(['location_id'])['Rewards_sales','Rewards_trans','Non_rewards_sales','Non_rewards_trans'].sum().reset_index()
cum_daily_summary_sales_trans=pd.merge(cum_daily_summary_sales_trans,cum_shopper_2018_count,on="location_id",how="inner")
cum_daily_summary_sales_trans['cum_weeks']=Cum_nth_week
cum_daily_summary_sales_trans=pd.merge(cum_daily_summary_sales_trans,df_store_quadrant,on="location_id",how="outer")

In [14]:
output_rewards_table=cum_daily_summary_sales_trans[['location_id','Rewards_sales','Rewards_trans','Non_rewards_sales','Non_rewards_trans','shoppers_cum_2018Q4','cum_weeks','Quadrant']]
output_rewards_table_store_count=output_rewards_table.groupby(['Quadrant','cum_weeks'])['location_id'].count().to_frame().reset_index().rename(columns={"location_id":"store_count"})
output_rewards_table_sum=output_rewards_table.groupby(['Quadrant','cum_weeks'])['Rewards_sales','Rewards_trans','Non_rewards_sales','Non_rewards_trans','shoppers_cum_2018Q4'].sum().reset_index()
output_rewards_table=pd.merge(output_rewards_table_store_count,output_rewards_table_sum,on=["Quadrant",'cum_weeks'],how="outer")

output_rewards_table['Avg_Rewards_Shoppers_per_store']=output_rewards_table['shoppers_cum_2018Q4']/output_rewards_table['store_count']
output_rewards_table['Transaction_per_Rewards_Shopper']=output_rewards_table['Rewards_trans']/output_rewards_table['shoppers_cum_2018Q4']
output_rewards_table['Sales_per_Rewards_Shopper']=output_rewards_table['Rewards_sales']/output_rewards_table['shoppers_cum_2018Q4']
output_rewards_table['Rewards_Avg_Order_Value']=output_rewards_table['Rewards_sales']/output_rewards_table['Rewards_trans']
output_rewards_table['Rewards_Sales_Share']=output_rewards_table['Rewards_sales']/(output_rewards_table['Rewards_sales']+output_rewards_table['Non_rewards_sales'])

output_rewards_table['Rewards_Sales_Share']=output_rewards_table['Rewards_Sales_Share'].apply(lambda x: np.round(x,4))
output_rewards_table['Sales_per_Rewards_Shopper']=output_rewards_table['Sales_per_Rewards_Shopper'].apply(lambda x: np.round(x,2))
output_rewards_table['Transaction_per_Rewards_Shopper']=output_rewards_table['Transaction_per_Rewards_Shopper'].apply(lambda x: np.round(x,4))
output_rewards_table['Avg_Rewards_Shoppers_per_store']=output_rewards_table['Avg_Rewards_Shoppers_per_store'].apply(lambda x: np.round(x,0)).astype(int)
output_rewards_table['Rewards_Avg_Order_Value']=output_rewards_table['Rewards_Avg_Order_Value'].apply(lambda x: np.round(x,2))
#Non Rewards also in the df above
output_1_rewards_table=output_rewards_table[['Quadrant','cum_weeks','store_count','Avg_Rewards_Shoppers_per_store','Transaction_per_Rewards_Shopper','Sales_per_Rewards_Shopper','Rewards_Avg_Order_Value','Rewards_Sales_Share','Rewards_sales']].rename(columns={'Rewards_sales':'Total_Rewards_Sales'})
output_1_rewards_table=output_1_rewards_table.sort_values(['Quadrant'])

rewards_share_df=output_1_rewards_table[['Quadrant','cum_weeks','store_count','Rewards_Sales_Share']]

In [15]:
#Non Rewards also in the df above
output_rewards_table['Non_Rewards_Transactions/Store']=output_rewards_table['Non_rewards_trans']/output_rewards_table['store_count']
output_rewards_table['Non_Rewards_Sales/Store']=output_rewards_table['Non_rewards_sales']/output_rewards_table['store_count']
output_rewards_table['Non_Rewards_Avg_Order_Value']=output_rewards_table['Non_rewards_sales']/output_rewards_table['Non_rewards_trans']
output_rewards_table['Non_Rewards_Sales_Share']=output_rewards_table['Non_rewards_sales']/(output_rewards_table['Rewards_sales']+output_rewards_table['Non_rewards_sales'])

output_rewards_table['Non_Rewards_Sales_Share']=output_rewards_table['Non_Rewards_Sales_Share'].apply(lambda x: np.round(x,4))
output_rewards_table['Non_Rewards_Transactions/Store']=output_rewards_table['Non_Rewards_Transactions/Store'].apply(lambda x: np.round(x,2))
output_rewards_table['Non_Rewards_Sales/Store']=output_rewards_table['Non_Rewards_Sales/Store'].apply(lambda x: np.round(x,2))
output_rewards_table['Non_Rewards_Avg_Order_Value']=output_rewards_table['Non_Rewards_Avg_Order_Value'].apply(lambda x: np.round(x,2))

output_2_Non_rewards_table=output_rewards_table[['Quadrant','cum_weeks','store_count','Non_Rewards_Transactions/Store','Non_Rewards_Sales/Store',
                                            'Non_Rewards_Avg_Order_Value','Non_Rewards_Sales_Share','Non_rewards_sales']].rename(columns={'Non_rewards_sales':'Total_Non_Rewards_Sales'})
output_2_Non_rewards_table=output_2_Non_rewards_table.sort_values(['Quadrant'])

In [16]:
output_3_total_Table=sales_all_Q4_inclusions_Cum_by_Quadrant[['Quadrant','cum_weeks','store_count','sales_2018','trans_2018']].rename(columns={"sales_2018":"Sales",'trans_2018':'Transactions'})
rewards_share_df=output_1_rewards_table[['Quadrant','cum_weeks','store_count','Rewards_Sales_Share']]
output_3_total_Table=pd.merge(output_3_total_Table,rewards_share_df,on=['Quadrant','cum_weeks','store_count'],how="outer")
output_3_total_Table['AOV']=(output_3_total_Table['Sales']/output_3_total_Table['Transactions']).apply(lambda x: np.round(x,2))
output_3_total_Table=output_3_total_Table[['Quadrant','cum_weeks','store_count','Sales','Transactions','AOV','Rewards_Sales_Share']]


In [17]:
cum_daily_summary_sales_trans.to_csv(write_folder+"cum_daily_sales.csv",index=False)
output_1_rewards_table.to_csv(write_folder+"output_1_rewards.csv",index=False)
output_2_Non_rewards_table.to_csv(write_folder+"output_2_Non_rewards.csv",index=False)
output_3_total_Table.to_csv(write_folder+"output_3_total_sales.csv",index=False)
df_checking_sales.to_csv(write_folder+"checking_0_daily_sales.csv",index=False)
sales_all_Q4_inclusions_Cum.to_csv(write_folder+"Cum_by_Store.csv",index=False)